In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains.summarize import load_summarize_chain

from langchain_openai import ChatOpenAI,OpenAI
import langchain 
langchain.debug=True
loader = PyPDFLoader("4.pdf")
docs=loader.load()
from langchain_text_splitters import RecursiveCharacterTextSplitter,CharacterTextSplitter


# text_splitter = RecursiveCharacterTextSplitter(
#     # Set a really small chunk size, just to show.
#     chunk_size=50000,
#     chunk_overlap=1000,
# )

# docs=text_splitter.split_documents(docs)



text_splitter = CharacterTextSplitter(
    separator="\n", chunk_size=100000, chunk_overlap=100, length_function=len)


result = '\n'.join(doc.page_content for doc in docs)
print(len(result))
docs = text_splitter.split_text(result)
from dotenv import load_dotenv
import os

from langchain_core.documents import Document

temp_doc=[Document(page_content=doc) for doc in docs]
len(docs)
docs=temp_doc
load_dotenv(override=True)


api_key = os.getenv('open_ai_key')



llm = ChatOpenAI(openai_api_key=api_key,model="gpt-4o-mini", temperature=0)

1046434


In [2]:
from langchain.chains import RefineDocumentsChain, LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import OpenAI

# Initialize the LLM (assuming you have set up the OpenAI instance)

def generatequestions(question):
    from langchain.chains import RefineDocumentsChain, LLMChain
    from langchain_core.prompts import PromptTemplate
    from langchain_community.llms import OpenAI


    print(question)

    # Define the prompt for summarizing the document and answering the specific questions
    initial_prompt = PromptTemplate.from_template(
        "Please summarize the following document and answer the following questions based on the summary:\n"
        f"{question}"
       
        "Document content:\n{context}"
    )
    initial_llm_chain = LLMChain(llm=llm, prompt=initial_prompt)
    initial_response_name = "prev_response"

    # Define the prompt for refining the answers and ensuring comprehensiveness
    refine_prompt = PromptTemplate.from_template(
        "Here are the initial answers to the questions based on the document summary: {prev_response}. "
        "Keep Summarizing the document apart from answering the question"
        "Refine these answers to ensure they are accurate and comprehensive based on the additional context provided:\n"
        f"{question}"
        "Additional context:\n{context}"
    )
    refine_llm_chain = LLMChain(llm=llm, prompt=refine_prompt)

    # Define the document prompt for formatting
    document_prompt = PromptTemplate(
        input_variables=["page_content"],
        template="{page_content}"
    )
    document_variable_name = "context"

    # Create the RefineDocumentsChain with the updated chains and prompts
    chain = RefineDocumentsChain(
        initial_llm_chain=initial_llm_chain,
        refine_llm_chain=refine_llm_chain,
        document_prompt=document_prompt,
        document_variable_name=document_variable_name,
        initial_response_name=initial_response_name,
    )
    result=chain.run(docs)
    #k="Answer is the question itself which is "+ question
   
    return result



In [3]:
from langchain.agents import Tool
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent

#ChatGoogleGenerativeAI(google_api_key=api_key,model="gemini-1.5-flash-latest", temperature=0.3)
import os
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain



from pydantic.v1 import BaseModel, Field
class DocumentInput(BaseModel):
    question: str = Field(description="Question that user asked, as it is")
class BriefInput(BaseModel):
    question: str = Field(description="User query, as it is")
abc=  Tool(
    args_schema=DocumentInput,
    name="FunctionHi",
    description=f"Please use this function to answer any type of questions and response according with the function",
    func=generatequestions
)

def usersay(question):
    k=llm.invoke(question)


    return k.content

test=  Tool(
    args_schema=BriefInput,
    name="FunctionHi",
    description=f"Use this to create a brief",
    func=usersay
)


from langchain.agents import AgentType, initialize_agent
import os

tools=[]
tools.append(abc)
print(tools)


[Tool(name='FunctionHi', description='Please use this function to answer any type of questions and response according with the function', args_schema=<class '__main__.DocumentInput'>, func=<function generatequestions at 0x00000167D9F70B80>)]


In [4]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

def create_agent(llm: ChatOpenAI, tools: list, system_prompt: str):
    # Each worker node will be given a name and some tools.
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

def create_agent2(llm: ChatOpenAI, system_prompt: str):
    # Each worker node will be given a name and some tools.
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_openai_tools_agent(llm, prompt)
    executor = AgentExecutor(agent=agent)
    return executor
def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

In [5]:
from langchain_core.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

members = ["LegalResearcher", "BriefEditor"]
system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    " following workers:  {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
)
# Our team supervisor is an LLM node. It just picks the next agent to process
# and decides when the work is completed
options = ["FINISH"] + members
# Using openai function calling can make output parsing easier for us
function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "next": {
                "title": "Next",
                "anyOf": [
                    {"enum": options},
                ],
            }
        },
        "required": ["next"],
    },
}
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should act next?"
            " Or should we FINISH? Select one of: {options}",
        ),
    ]
).partial(options=str(options), members=", ".join(members))

# llm = ChatOpenAI(model="gpt-4-1106-preview")

supervisor_chain = (
    prompt
    | llm.bind_functions(functions=[function_def], function_call="route")
    | JsonOutputFunctionsParser()
)

In [6]:
import functools
import operator
from typing import Sequence, TypedDict
from typing import Annotated, Sequence


from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langgraph.graph import END, StateGraph, START


# The agent state is the input to each node in the graph
class AgentState(TypedDict):
    # The annotation tells the graph that new messages will always
    # be added to the current states
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # The 'next' field indicates where to route to next
    next: str


research_agent = create_agent(llm, [abc], "You are a lawyer who will research on user query and invoke tool to get the answer you want, but it should never be invoked for any request of brief creation or draft")
research_node = functools.partial(agent_node, agent=research_agent, name="LegalResearcher")

# NOTE: THIS PERFORMS ARBITRARY CODE EXECUTION. PROCEED WITH CAUTION
comedian_agent = create_agent(
    llm,
    [test],
    "You will create brief as US brief's are made, and it will be only invoked when user wants to write brief ",
)
comedian_node = functools.partial(agent_node, agent=comedian_agent, name="BriefEditor")

workflow = StateGraph(AgentState)
workflow.add_node("LegalResearcher", research_node)
workflow.add_node("BriefEditor", comedian_node)
workflow.add_node("supervisor", supervisor_chain)

In [7]:
for member in members:
    # We want our workers to ALWAYS "report back" to the supervisor when done
    workflow.add_edge(member, "supervisor")
# The supervisor populates the "next" field in the graph state
# which routes to a node or finishes
conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)
# Finally, add entrypoint
workflow.add_edge(START, "supervisor")

graph = workflow.compile()

In [65]:
# final_state = graph.invoke(
#     {"messages": [HumanMessage(content="Suppose, my client is blind, is there any law that can help her?")]},
#     config={"configurable": {"thread_id": 42}}
# )

[chain/start] [chain:LangGraph] Entering Chain run with input:
[inputs]
[chain/start] [chain:LangGraph > chain:__start__] Entering Chain run with input:
[inputs]
[chain/end] [chain:LangGraph > chain:__start__] s] Exiting Chain run with output:
[outputs]
[chain/start] [chain:LangGraph > chain:supervisor] Entering Chain run with input:
[inputs]
[chain/start] [chain:LangGraph > chain:supervisor > prompt:ChatPromptTemplate] Entering Prompt run with input:
[inputs]
[chain/end] [chain:LangGraph > chain:supervisor > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:LangGraph > chain:supervisor > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: You are a supervisor tasked with managing a conversation between the following workers:  LegalResearcher, BriefEditor. Given the following user request, respond with the worker to act next. Each worker will perform a task and respond with their results and status. When finished, respond 

In [ ]:
print(final_state["messages"][-1].content)

Yes, there are several laws and rulings that can assist your blind client, particularly in the context of disability claims. Here are some key points:

1. **Social Security Rulings (SSR)**: SSR 18-02p specifically addresses blindness claims, ensuring that individuals with visual impairments receive appropriate consideration in the disability determination process.

2. **Eligibility for Social Security Disability Benefits**: To qualify, individuals must have a medically determinable impairment that significantly limits their ability to perform basic work activities.

3. **Failure to Follow Prescribed Treatment**: If an individual has not followed prescribed treatment, the Social Security Administration (SSA) will evaluate whether adherence would have restored their ability to engage in substantial gainful activity (SGA). Acceptable reasons for not following treatment can include financial constraints or medical disagreements.

4. **Past Relevant Work (PRW)**: The evaluation period for c

In [8]:
final_state = graph.invoke(
    {"messages": [HumanMessage(content="You are a US Lawyer, and your client is blind, for her, you have to write a brief draft and submit in the court")]},
    config={"configurable": {"thread_id": 42}}
)

[chain/start] [chain:LangGraph] Entering Chain run with input:
[inputs]
[chain/start] [chain:LangGraph > chain:__start__] Entering Chain run with input:
[inputs]
[chain/end] [chain:LangGraph > chain:__start__] s] Exiting Chain run with output:
[outputs]
[chain/start] [chain:LangGraph > chain:supervisor] Entering Chain run with input:
[inputs]
[chain/start] [chain:LangGraph > chain:supervisor > prompt:ChatPromptTemplate] Entering Prompt run with input:
[inputs]
[chain/end] [chain:LangGraph > chain:supervisor > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:LangGraph > chain:supervisor > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: You are a supervisor tasked with managing a conversation between the following workers:  LegalResearcher, BriefEditor. Given the following user request, respond with the worker to act next. Each worker will perform a task and respond with their results and status. When finished, respond 

In [9]:
print(final_state["messages"][-1].content)

Here is a draft of a legal brief for your blind client, focusing on accessibility rights and discrimination under the Americans with Disabilities Act (ADA):

---

**IN THE UNITED STATES DISTRICT COURT FOR THE [INSERT DISTRICT]**

**[CLIENT NAME],**  
Plaintiff,  

v.  

**[DEFENDANT NAME],**  
Defendant.  

**Case No. [INSERT CASE NUMBER]**  

**PLAINTIFF'S LEGAL BRIEF IN SUPPORT OF CLAIM UNDER THE AMERICANS WITH DISABILITIES ACT**

**I. INTRODUCTION**

This brief is submitted on behalf of Plaintiff [Client Name], a blind individual, who brings this action against Defendant [Defendant Name] for violations of the Americans with Disabilities Act (ADA), 42 U.S.C. § 12101 et seq. The Plaintiff asserts that the Defendant has failed to provide adequate accessibility measures, resulting in discrimination against the Plaintiff and other individuals with disabilities.

**II. FACTS OF THE CASE**

1. **Background of the Plaintiff**: Plaintiff [Client Name] is a legally blind individual who relies

In [ ]:
k=llm.invoke("Write a brief in US legal terms for diabled person")

[llm/start] [llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Write a brief in US legal terms for diabled person"
  ]
}
[llm/end] [llm:ChatOpenAI] [7.75s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "**Legal Brief: Rights and Protections for Individuals with Disabilities**\n\n**I. Introduction**\n\nThis brief outlines the legal rights and protections afforded to individuals with disabilities under federal and state law in the United States. It aims to provide a comprehensive overview of relevant statutes, regulations, and case law that safeguard the rights of disabled persons.\n\n**II. Relevant Legislation**\n\n1. **Americans with Disabilities Act (ADA)**\n   - Enacted in 1990, the ADA prohibits discrimination against individuals with disabilities in various areas, including employment, public accommodations, transportation, and telecommunications.\n   - Title I of the ADA addresses employment discrimination, requiring employ

In [ ]:
print(k.content)

**Legal Brief: Rights and Protections for Individuals with Disabilities**

**I. Introduction**

This brief outlines the legal rights and protections afforded to individuals with disabilities under federal and state law in the United States. It aims to provide a comprehensive overview of relevant statutes, regulations, and case law that safeguard the rights of disabled persons.

**II. Relevant Legislation**

1. **Americans with Disabilities Act (ADA)**
   - Enacted in 1990, the ADA prohibits discrimination against individuals with disabilities in various areas, including employment, public accommodations, transportation, and telecommunications.
   - Title I of the ADA addresses employment discrimination, requiring employers to provide reasonable accommodations to qualified individuals with disabilities.

2. **Rehabilitation Act of 1973**
   - Section 504 of the Rehabilitation Act prohibits discrimination against individuals with disabilities in programs and activities receiving federal 